online

if random.choice([0,1,1,1,]):
[0.859, 0.86, 0.861, 0.843, 0.855] 0.8556000000000001

[0.8789124971441612, 0.8831880936061866, 0.8818342151675468, 0.8683454177281323, 0.8759553203997628] 0.877647108809158

online 0.7916984126984127

# 两个网络

In [1]:
seed = 0
import random
import numpy as np
import tensorflow as tf
import os
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
os.environ["CUDA_DEVICE_ORDER"] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
os.environ["PYTHONHASHSEED"] = str(seed)

os.environ['TF_DETERMINISTIC_OPS'] = '1'

# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     try:
#         tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=7000)])
#     except RuntimeError as e:
#         print(e)

In [2]:
import numpy as np
import pandas as pd
# 选择比较好的模型
import seaborn as sns

import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.signal import resample
from tensorflow.keras import layers
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import os
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from tensorflow.keras.losses import categorical_crossentropy

from sklearn.preprocessing import StandardScaler,MinMaxScaler
%load_ext autoreload
%autoreload 2

def acc_combo(y, y_pred):
    # 数值ID与行为编码的对应关系
    mapping = {
        '0': 'A_1', '1': 'B_2', '2': 'A_3', '3': 'A_4', '4': 'B_3', '5': 'C_5', '6': 'C_2', '7': 'A_5', '8': 'B_1', 
        '9': 'C_1', '10': 'A_2', '11': 'C_3', '12': 'B_5', '13': 'B_4', '14': 'C_4', 
        '15': 'D_6', '16': 'E_7', '17': 'F_8', '18': 'G_9', '19': 'H_0'
              }
    # 将行为ID转为编码
    code_y, code_y_pred = mapping[str(int(y))], mapping[str(int(y_pred))]
    if code_y == code_y_pred: #编码完全相同得分1.0
        return 1.0
    elif code_y.split("_")[0] == code_y_pred.split("_")[0]: #编码仅字母部分相同得分1.0/7
        return 1.0/7
    elif code_y.split("_")[1] == code_y_pred.split("_")[1]: #编码仅数字部分相同得分1.0/3
        return 1.0/3
    else:
        return 0.0

sample_num = 60

In [3]:
!ls ../../data/final_data

sensor_test_final.csv  sensor_train_final.csv  submit_example.csv


In [4]:
root_path  = '../../data/final_data/'
train = pd.read_csv(root_path+'sensor_train_final.csv')
test = pd.read_csv(root_path+'sensor_test_final.csv')
sub = pd.read_csv(root_path+'submit_example.csv')
y = train.groupby('fragment_id')['behavior_id'].min()

In [5]:
def add_features(df):
    print(df.columns)
    df['acc'] = (df.acc_x ** 2 + df.acc_y ** 2 + df.acc_z ** 2) ** .5
    df['accg'] = (df.acc_xg ** 2 + df.acc_yg ** 2 + df.acc_zg ** 2) ** .5
    df['thetax']=np.arctan(df.acc_xg/
                           np.sqrt(df.acc_yg*df.acc_yg+df.acc_zg*df.acc_zg))*180/np.pi
    df['thetay']=np.arctan(df.acc_yg/
                           np.sqrt(df.acc_xg*df.acc_xg+df.acc_zg*df.acc_zg))*180/np.pi
    df['thetaz']=np.arctan(df.acc_zg/
                           np.sqrt(df.acc_yg*df.acc_yg+df.acc_xg*df.acc_xg))*180/np.pi

    df['xy'] = (df['acc_x'] ** 2 + df['acc_y'] ** 2) ** 0.5
    df['xy_g'] = (df['acc_xg'] ** 2 + df['acc_yg'] ** 2) ** 0.5    
    
    df['g'] = ((df["acc_x"] - df["acc_xg"]) ** 2 + 
                 (df["acc_y"] - df["acc_yg"]) ** 2 + (df["acc_z"] - df["acc_zg"]) ** 2) ** 0.5

    print(df.columns)
    return df
train=add_features(train)
test=add_features(test)

Index(['fragment_id', 'time_point', 'acc_x', 'acc_y', 'acc_z', 'acc_xg',
       'acc_yg', 'acc_zg', 'behavior_id'],
      dtype='object')
Index(['fragment_id', 'time_point', 'acc_x', 'acc_y', 'acc_z', 'acc_xg',
       'acc_yg', 'acc_zg', 'behavior_id', 'acc', 'accg', 'thetax', 'thetay',
       'thetaz', 'xy', 'xy_g', 'g'],
      dtype='object')
Index(['fragment_id', 'time_point', 'acc_x', 'acc_y', 'acc_z', 'acc_xg',
       'acc_yg', 'acc_zg'],
      dtype='object')
Index(['fragment_id', 'time_point', 'acc_x', 'acc_y', 'acc_z', 'acc_xg',
       'acc_yg', 'acc_zg', 'acc', 'accg', 'thetax', 'thetay', 'thetaz', 'xy',
       'xy_g', 'g'],
      dtype='object')


In [6]:
group1 = [x for x in train.columns if x not in ['fragment_id', 'time_point','behavior_id']]
print(group1)

train = train[['fragment_id', 'time_point', 'behavior_id']+group1]
test = test[['fragment_id', 'time_point']+group1]

['acc_x', 'acc_y', 'acc_z', 'acc_xg', 'acc_yg', 'acc_zg', 'acc', 'accg', 'thetax', 'thetay', 'thetaz', 'xy', 'xy_g', 'g']


In [7]:
FEATURE_NUM=14

In [8]:
len(set(train['fragment_id'])),len(set(test['fragment_id']))

(15000, 16000)

In [9]:

x = np.zeros((15000, sample_num, FEATURE_NUM, 1))
t = np.zeros((16000, sample_num, FEATURE_NUM, 1))

In [10]:
def df2array(df,num,x):
    for i in tqdm(range(num)):
        tmp = df[df.fragment_id == i][:sample_num]
        length=len(tmp)
        if length<50:
            new_tmp=tmp.copy()
            while len(tmp)<60:
                new_tmp['time_point']=tmp['time_point'].max()+new_tmp['time_point']
                tmp=pd.concat([tmp,new_tmp],ignore_index=True)
        tmp=tmp.iloc[:60,:]
        x[i,:,:,0] = resample(tmp[group1], sample_num, np.array(tmp.time_point))[0].reshape(sample_num,FEATURE_NUM)
    return x
x=df2array(train,15000,x)
t=df2array(test,16000,t)

100%|██████████| 16000/16000 [00:48<00:00, 329.55it/s]


In [15]:
%%time
# 一个完成了的generator
def data_generator(data,label,class17label,batch_size):
    """
    data:array  (7292, 60, 14, 1)
    label:array (7292,)
    class17label: series
    """
    class17label=np.asarray(class17label)
    length=len(data)
    seq_length=len(data[0])
    half_seq_length=int(seq_length/2)
    
    # index2label
    index2label=dict(zip(range(length),class17label))
    
    label2index={}
#     print(class17label)
    for i in range(length):
#         print(class17label[i],label2index.get(class17label[i],[]))
        label2index[class17label[i]]=label2index.get(class17label[i],[])
        label2index[class17label[i]].append(i)

    count=0
    np.random.seed(seed)# 保证结果可重复
    
    while True:
        
        if count==0 or (count + 1) * batch_size > length:  # 如果是第一个或者最后一个batch
            count=0
            shuffle_index = list(range(length))
            np.random.shuffle(shuffle_index)   ## 对索引进行打乱
        
        start = count * batch_size  ## batch的起始点
        end = (count + 1) * batch_size ## batch的终点
        inds=shuffle_index[start:end]

        count+=1
        
        if random.choice([0,1,1,1]):
            # minxup
            #one specific index -> label -> all the index belong to this
            choice_index=[random.choice(label2index[index2label[x]]) for x in inds]   # get the random choice seq(waiting for concat)
            # 1st 前1/2 seq_length 点原始  后1/2 seq_length 点随机
            res_x_orig=data[inds,:half_seq_length]   #原始
            res_x=data[choice_index,half_seq_length:]   #需要加入的

    #         print(inds)
    #         print(data.shape,res_x_orig.shape,res_x.shape,np.concatenate((res_x_orig,res_x),axis=1).shape)
            yield np.concatenate((res_x_orig,res_x),axis=1),\
                    [label[0][inds],label[1][inds],label[2][inds]]
        else:
        
            yield data[inds],[label[0][inds],label[1][inds],label[2][inds]]
            
    

count=0
for a,b in data_generator(x,[y,y,y],y,32):
    print(a.shape,b[0].shape)
    count+=1
    if count==20:
        break


(32, 60, 14, 1) (32,)
(32, 60, 14, 1) (32,)
(32, 60, 14, 1) (32,)
(32, 60, 14, 1) (32,)
(32, 60, 14, 1) (32,)
(32, 60, 14, 1) (32,)
(32, 60, 14, 1) (32,)
(32, 60, 14, 1) (32,)
(32, 60, 14, 1) (32,)
(32, 60, 14, 1) (32,)
(32, 60, 14, 1) (32,)
(32, 60, 14, 1) (32,)
(32, 60, 14, 1) (32,)
(32, 60, 14, 1) (32,)
(32, 60, 14, 1) (32,)
(32, 60, 14, 1) (32,)
(32, 60, 14, 1) (32,)
(32, 60, 14, 1) (32,)
(32, 60, 14, 1) (32,)
(32, 60, 14, 1) (32,)
CPU times: user 38.6 ms, sys: 1.97 ms, total: 40.5 ms
Wall time: 38.1 ms


In [16]:
def ConvBNRelu(X,filters,kernal_size=(3,3)):
    X = Conv2D(filters=filters,
               kernel_size=kernal_size,
#                activation='relu',
               use_bias=False,
               padding='same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    return X


def ConvRelu(X,filters,kernal_size=(3,3)):
    X = Conv2D(filters=filters,
               kernel_size=kernal_size,
               activation='relu',
               use_bias=False,
               padding='same')(X)
    return X


def squeeze_excitation_layer(x, out_dim,ratio=8):
    '''
    SE module performs inter-channel weighting.
    '''
    squeeze = GlobalAveragePooling2D()(x)

    excitation = Dense(units=out_dim // ratio)(squeeze)
    excitation = Activation('relu')(excitation)
    excitation = Dense(units=out_dim)(excitation)
    excitation = Activation('sigmoid')(excitation)
    excitation = Reshape((1,1,out_dim))(excitation)
    scale = multiply([x,excitation])
    return scale

# def SE_Residual(X):
#     A = 
#     X = squeeze_excitation_layer(X,128)
#     X =  Add()([X,A])
    

def lenet5(input):
    A = ConvBNRelu(input,64,kernal_size=(3,3))
#     B = ConvBNRelu(input,16,kernal_size=(5,1))
#     C = ConvBNRelu(input,16,kernal_size=(7,1))
#     ABC = layers.Concatenate()([A,B,C])
    X = ConvBNRelu(A,128)
#     X = squeeze_excitation_layer(X,128)
    X = Dropout(0.2)(X)

    X = AveragePooling2D()(X)
    
    X = ConvBNRelu(X,256)
    X = Dropout(0.3)(X)
#     X = squeeze_excitation_layer(X,256)
    X = ConvBNRelu(X,512)   
    X = Dropout(0.5)(X)
#     X = squeeze_excitation_layer(X,512)
#     X = GlobalMaxPooling2D()(X)
    X = GlobalAveragePooling2D()(X)
    
#     X = BatchNormalization()(X)
    return X
import tensorflow as tf
def Net(sample_num):
    input1 = Input(shape=(sample_num, FEATURE_NUM, 1))
    part = tf.split(input1,axis=2, num_or_size_splits = [6, 2, 6])
#     res = tf.split(c, axis = 3, num_or_size_splits = [2, 2, 4])
    
    #######loss1
    X1 = Concatenate(axis=-2)([part[0],part[1]])
    X1 = lenet5(X1)
    X1 = BatchNormalization()(X1)
    X1 = Dense(128, activation='relu')(X1)
    X1 = BatchNormalization()(X1)
    X1 = Dropout(0.2)(X1)

    X2 = Concatenate(axis=-2)([part[0],part[2]])
    X2 = lenet5(X2)    
    X2 = BatchNormalization()(X2)
#     X = Dense(512, activation='relu')(X)
#     X = BatchNormalization()(X)
    X2 = Dense(128, activation='relu')(X2)
    X2 = BatchNormalization()(X2)
    X2 = Dropout(0.2)(X2)
    X12 = Concatenate(axis=-1)([X1,X2])
    
    output1 = Dense(8, activation='softmax', name='4class')(X12)   # 大类-字母
    
    #######loss2
    X3 = Concatenate(axis=-2)([part[0],part[1]])
    X3 = lenet5(X3)
    X3 = BatchNormalization()(X3)
    X3 = Dense(128, activation='relu')(X3)
    X3 = BatchNormalization()(X3)
    X3 = Dropout(0.2)(X3)

    X4 = Concatenate(axis=-2)([part[0],part[2]])
    X4 = lenet5(X4)    
    X4 = BatchNormalization()(X4)
#     X = Dense(512, activation='relu')(X)
#     X = BatchNormalization()(X)
    X4 = Dense(128, activation='relu')(X4)
    X4 = BatchNormalization()(X4)
    X4 = Dropout(0.2)(X4)
    
    X34 = Concatenate(axis=-1)([X3,X4])
    
    output2 = Dense(10, activation='softmax', name='7class')(X34)   # 大类-数字
    
    X = Concatenate(axis=-1)([X12,X34])
    
    X = Dense(64)(X)
    output3 = Dense(20, activation='softmax',name='19class')(X) #小类
    
    
    return Model([input1], [output1,output2,output3])

model = Net(60)
model.summary()

In [13]:
# 两个输出    
mapping = {
    '0': 'A_1', '1': 'B_2', '2': 'A_3', '3': 'A_4', '4': 'B_3', '5': 'C_5', '6': 'C_2', '7': 'A_5', '8': 'B_1', 
    '9': 'C_1', '10': 'A_2', '11': 'C_3', '12': 'B_5', '13': 'B_4', '14': 'C_4', 
    '15': 'D_6', '16': 'E_7', '17': 'F_8', '18': 'G_9', '19': 'H_0'
          }

# 每一个大类输出 4
new_mapping = {'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7}

from sklearn.utils.class_weight import compute_class_weight
# # y_train_weight = compute_sample_weight("balanced", train['behavior_id'])
classweights1=compute_class_weight("balanced",['A','B','C','D','E','F','G','H'],\
                                   pd.read_csv(root_path+'sensor_train_final.csv')['behavior_id'].apply(lambda x:mapping[str(x)][0]))
classweights1=pd.DataFrame(classweights1)[0].to_dict()
# classweights1


tmp=pd.read_csv(root_path+'sensor_train_final.csv')['behavior_id'].apply(lambda x:mapping[str(x)][2])
tmp=[int(x) for x in tmp]
classweights2=compute_class_weight("balanced",list(range(10)),tmp)
classweights2=pd.DataFrame(classweights2)[0].to_dict()
classweights2

# y_train_weight = compute_sample_weight("balanced", train['behavior_id'])
classweights3=compute_class_weight("balanced",np.array(range(20)), pd.read_csv(root_path+'sensor_train_final.csv')['behavior_id'])
classweights3=pd.DataFrame(classweights3)[0].to_dict()
classweights1,classweights2,classweights3

/home/team_04/anaconda3/envs/tf2_torch/lib/python3.7/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass classes=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'], y=0         C
1         C
2         C
3         C
4         C
         ..
391789    E
391790    E
391791    E
391792    E
391793    E
Name: behavior_id, Length: 391794, dtype: object as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/home/team_04/anaconda3/envs/tf2_torch/lib/python3.7/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass classes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], y=[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 6, 6, 6, 6, 6, 

({0: 0.5414510779436152,
  1: 0.43855835445191677,
  2: 0.5746600096218157,
  3: 2.7533732501264967,
  4: 2.6801428336890494,
  5: 4.144038754442376,
  6: 2.462626338814301,
  7: 1.3349938667030121},
 {0: 1.0679950933624096,
  1: 0.7043614267222782,
  2: 1.2670396481469504,
  3: 0.7505200850525832,
  4: 0.7652824439409329,
  5: 0.40225669668066405,
  6: 2.2026986001011974,
  7: 2.1441142669512394,
  8: 3.315231003553901,
  9: 1.9701010710514406},
 {0: 0.9323101085094232,
  1: 1.2855820973881087,
  2: 1.3505480868665978,
  3: 1.3001725625539258,
  4: 0.9258329788742379,
  5: 0.6896567505720824,
  6: 2.068606124604013,
  7: 0.5821258766195174,
  8: 0.9612217860647694,
  9: 1.376454468802698,
  10: 3.152510460251046,
  11: 1.184454924723381,
  12: 0.5542895138928188,
  13: 1.0019281914893616,
  14: 1.1817397599083066,
  15: 1.1013493000505987,
  16: 1.0720571334756197,
  17: 1.6576155017769505,
  18: 0.9850505355257203,
  19: 0.5339975466812048})

In [14]:
# [:,:,:,[1]]
train = x
test = t

    
fold_num=5

proba_t = np.zeros((16000, 20))
proba_oof = np.zeros((15000,20))

oof_score = []
oof_comm = []
history = []

from tensorflow.keras.losses import categorical_crossentropy
def custom_loss(y_true, y_pred):
    return categorical_crossentropy(y_true, y_pred, label_smoothing=0.05)

# 两个输出    
mapping = {
    0: 'A_1', 1: 'B_2', 2: 'A_3', 3: 'A_4', 4: 'B_3', 5: 'C_5', 6: 'C_2', 7: 'A_5', 8: 'B_1', 
    9: 'C_1', 10: 'A_2', 11: 'C_3', 12: 'B_5', 13: 'B_4', 14: 'C_4', 
    15: 'D_6', 16: 'E_7', 17: 'F_8',18: 'G_9', 19: 'H_0'
          }
# 每一个大类输出 4
new_mapping = {'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7}
y_1 = to_categorical([new_mapping[mapping[x][0]] for x in y], num_classes=8)
# 每一个大类输出 
y_2 = to_categorical([mapping[x][2] for x in y], num_classes=10)
# 每一个小类的输出 19
y_3 = to_categorical(y, num_classes=20)

# seeds=[0,42,2020,666,888,6699,6969]
seeds=[42]
for seed in seeds:
    kfold = StratifiedKFold(fold_num,random_state=seed,shuffle=True)
    
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    
    for fold, (xx, yy) in enumerate(kfold.split(train, y)):

        model = Net(60)
        model.summary()
        model.compile(loss=[custom_loss,custom_loss,custom_loss],loss_weights=[3,7,21],
                      optimizer=Adam(),
                      metrics=["acc"])#'',localscore

        plateau3 = ReduceLROnPlateau(monitor="19class_acc",
                                    verbose=1,
                                    mode='max',
                                    factor=0.5,
                                    patience=18)
        early_stopping = EarlyStopping(monitor="val_19class_acc",
                                       verbose=1,
                                       mode='max',
                                       patience=60)

        checkpoint = ModelCheckpoint(f'all_stragety{fold}.h5',
                                     monitor="val_19class_acc",
                                     verbose=0,
                                     mode='max',
                                     save_best_only=True)

        train_res = model.fit_generator(data_generator(train[xx], [y_1[xx], y_2[xx], y_3[xx]],y[xx],512),
                  epochs=1000,
                  steps_per_epoch=len(xx) // 512,
                  verbose=1,
                  shuffle=True,
                  validation_data=(train[yy], [y_1[yy], y_2[yy],y_3[yy]]),
                  callbacks=[plateau3, early_stopping, checkpoint],
                             class_weight=[classweights1,classweights2,classweights3])

        history.append(train_res)

        model.load_weights(f'all_stragety{fold}.h5')
        proba_t += model.predict(test, verbose=0, batch_size=1024)[2] / (fold_num*len(seeds))
        proba_oof[yy] += model.predict(train[yy],verbose=0,batch_size=1024) [2]

        oof_y = np.argmax(proba_oof[yy], axis=1)
        acc = round(accuracy_score(y[yy], oof_y),3)
        print(acc)
        oof_score.append(acc)
        scores = sum(acc_combo(y_true, y_pred) for y_true, y_pred in zip(y[yy], oof_y)) / oof_y.shape[0]
        oof_comm.append(scores)   
        print(round(scores, 5))


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 60, 14, 1)]  0                                            
__________________________________________________________________________________________________
tf_op_layer_split_1 (TensorFlow [(None, 60, 6, 1), ( 0           input_2[0][0]                    
__________________________________________________________________________________________________
concatenate_7 (Concatenate)     (None, 60, 8, 1)     0           tf_op_layer_split_1[0][0]        
                                                                 tf_op_layer_split_1[0][1]        
__________________________________________________________________________________________________
concatenate_8 (Concatenate)     (None, 60, 12, 1)    0           tf_op_layer_split_1[0][0]  

23/23 [==============================] - 20s 856ms/step - loss: 67.5681 - 4class_loss: 1.6840 - 7class_loss: 1.6833 - 19class_loss: 2.4158 - 4class_acc: 0.3675 - 7class_acc: 0.4308 - 19class_acc: 0.3661 - val_loss: 283.7465 - val_4class_loss: 4.2155 - val_7class_loss: 4.1390 - val_19class_loss: 11.5299 - val_4class_acc: 0.1283 - val_7class_acc: 0.1027 - val_19class_acc: 0.1027
Epoch 2/1000
23/23 [==============================] - 14s 623ms/step - loss: 48.3130 - 4class_loss: 1.2938 - 7class_loss: 1.3465 - 19class_loss: 1.6670 - 4class_acc: 0.5027 - 7class_acc: 0.5346 - 19class_acc: 0.5099 - val_loss: 137.6222 - val_4class_loss: 3.6028 - val_7class_loss: 2.7625 - val_19class_loss: 5.1179 - val_4class_acc: 0.0500 - val_7class_acc: 0.1027 - val_19class_acc: 0.1027
Epoch 3/1000
23/23 [==============================] - 15s 637ms/step - loss: 47.4892 - 4class_loss: 1.2368 - 7class_loss: 1.3227 - 19class_loss: 1.6438 - 4class_acc: 0.5475 - 7class_acc: 0.5363 - 19class_acc: 0.5157 - val_loss: 

KeyboardInterrupt: 

In [ ]:
oof_score,np.mean(oof_score),oof_comm,np.mean(oof_comm)

In [1]:
for index,i in enumerate(oof_comm):
    print(index,i,oof_score[index])

oof_dict = {
    "oof":proba_oof,
    "test":proba_t,
    "acc":oof_comm,
}
import joblib 
joblib.dump(oof_dict,"all_stragety-add2net%.5f_dict.pkl"% np.mean(oof_comm))


NameError: name 'oof_comm' is not defined

In [ ]:
!ls ../data

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

def acc_combo(y, y_pred):
    # 数值ID与行为编码的对应关系
    mapping = {
        0: 'A_1', 1: 'B_2', 2: 'A_3', 3: 'A_4', 4: 'B_3', 5: 'C_5', 6: 'C_2', 7: 'A_5', 8: 'B_1', 
        9: 'C_1', 10: 'A_2', 11: 'C_3',12: 'B_5', 13: 'B_4', 14: 'C_4', 
        15: 'D_6', 16: 'E_7', 17: 'F_8', 18: 'G_9', 19: 'H_0'
              }
    # 将行为ID转为编码
    code_y, code_y_pred = mapping[y], mapping[y_pred]
    if code_y == code_y_pred: #编码完全相同得分1.0
        return 1.0
    elif code_y.split("_")[0] == code_y_pred.split("_")[0]: #编码仅字母部分相同得分1.0/7
        return 1.0/7
    elif code_y.split("_")[1] == code_y_pred.split("_")[1]: #编码仅数字部分相同得分1.0/3
        return 1.0/3
    else:
        return 0.0

train_y = y
labels = np.argmax(proba_t[:,:20], axis=1)
oof_y = np.argmax(proba_oof[:,:20], axis=1)
print(round(accuracy_score(train_y, oof_y), 5))
scores = sum(acc_combo(y_true, y_pred) for y_true, y_pred in zip(train_y, oof_y)) / oof_y.shape[0]
print(round(scores, 5))
data_path = '../data/'
sub = pd.read_csv(data_path+'submit_example.csv')
sub['behavior_id'] = labels

vc = pd.Series(train_y).value_counts().sort_index()
sns.barplot(vc.index, vc.values)
plt.show()

vc = pd.Series(oof_y).value_counts().sort_index()
sns.barplot(vc.index, vc.values)
plt.show()

vc = sub['behavior_id'].value_counts().sort_index()
sns.barplot(vc.index, vc.values)
plt.show()
sub.to_csv('all_stragety-add2net%.5f.csv' % scores, index=False)
sub.info()
